In [1]:
# Import libs
import os
import datetime
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# Load dataset
# get data_dir for tranining
current_path = os.getcwd()

img_height = 224
img_width = 224
batch_size = 64

# set train dataset
train_data_dir = os.path.join(current_path, 'Vegetable_Images\\train')
train_ds = keras.utils.image_dataset_from_directory(
    train_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

# set validation dataset
val_data_dir = os.path.join(current_path, 'Vegetable_Images\\validation')
val_ds = keras.utils.image_dataset_from_directory(
    val_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

class_names = train_ds.class_names
class_num = len(class_names)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


In [3]:
# Normalize data
data_scaling = keras.layers.Rescaling(scale=1./255)

train_ds = train_ds.map( lambda x, y: (data_scaling(x), y))
val_ds = val_ds.map( lambda x, y: (data_scaling(x), y))

In [4]:
# Build model
VGG16_excludeTop = keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(img_height, img_width, 3)
)
VGG16_excludeTop.trainable = False

Top_layers = keras.layers.Flatten()(VGG16_excludeTop.output)
Top_layers = keras.layers.Dense(units=1024, activation='relu', use_bias=True)(Top_layers)
Top_layers = keras.layers.Dropout(rate=0.25)(Top_layers)
Top_layers = keras.layers.Dense(units=class_num, activation='softmax', use_bias=True)(Top_layers)

VGG16_Model = keras.Model(inputs=VGG16_excludeTop.input, outputs=Top_layers)

VGG16_Model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

VGG16_Model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    25,691,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │        15,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,421,199 (154.19 MB)

 Trainable params: 25,706,511 (98.06 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [5]:
# Train model
logdir = os.path.join("logs", datetime.datetime.now().strftime("VGG16_Model-%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

history = VGG16_Model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tensorboard_callback]
)

Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1624s 7s/step - accuracy: 0.7716 - loss: 0.7902 - val_accuracy: 0.9873 - val_loss: 0.0591
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1613s 7s/step - accuracy: 0.9926 - loss: 0.0398 - val_accuracy: 0.9923 - val_loss: 0.0360
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1609s 7s/step - accuracy: 0.9993 - loss: 0.0116 - val_accuracy: 0.9937 - val_loss: 0.0257
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1607s 7s/step - accuracy: 0.9997 - loss: 0.0062 - val_accuracy: 0.9950 - val_loss: 0.0230
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1607s 7s/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.9957 - val_loss: 0.0203
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1602s 7s/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.9953 - val_loss: 0.0193
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1601s 7s/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.9950 - val_loss: 0.0190
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1603s 7s/step - accuracy: 1.0000 - loss: 0.0012 - 

In [6]:
# Save model as keras
VGG16_Model.save('saved_models\VGG16_Model.keras')